In [1]:
from os import getcwd
import os.path as op
from glob import glob

In [2]:
from clients.utils import create_allocation_table
from clients.expt_recipes.db_query import create_plates_from_allocation_table

expt_name = 'A81_E214'
allocation_table = create_allocation_table(expt_name)
all_expt_plates = create_plates_from_allocation_table(allocation_table)

In [3]:
from hardware.qpcr import QpcrDataFile

id_qpcr_plate = 'A81_E214_1_ID'
cwd = getcwd()
nav_root = op.abspath(op.join(cwd, op.pardir,
                              op.pardir, op.pardir))
data_folder = op.join(nav_root, 'hardware', 'tests', 'data')
f = op.join(data_folder, id_qpcr_plate + '.xls')

qpcr_data = QpcrDataFile(f)
id_qpcr_inst_plate = qpcr_data.get_data_by_well()

C:\Users\Prasanna.Gnanaraj\AppData\Local\Programs\Python\Python36\lib\site-packages\pandas\core\ops.py:816: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


In [4]:
from hardware.labchip import LabChip

lc_plate = '20180103_A'
fds = glob(op.join(data_folder, '*' + lc_plate + '*.csv'))

files = {}
for f in fds:
    if 'PeakTable' in f:
        files['peak_file'] = f
    elif 'RawTable' in f:
        files['raw_file'] = f
    elif 'WellTable' in f:
        
        files['well_file'] = f
lc = LabChip(**files)
lc_inst_plate = lc.get_data_by_well()

In [5]:
from clients.expt_recipes.db_query import get_assay_amplicon_length
from clients.expt_recipes.nested.model_builders import build_qpcr_constituents
from clients.expt_recipes.lost import build_labchip_datas_from_inst_data
from clients.expt_recipes.nested.constituents import IdConstituents
from clients.expt_recipes.nested.model_builders import (build_id_qpcr_datas_from_inst_data,
                                                         get_wells_by_id_assay)
from clients.expt_recipes.db_query import create_qpcr_lc_mapping, get_lc_dilutions
from clients.expt_recipes.nested.models import NestedMasterTable

In [6]:
%%time
id_plate_constituents = build_qpcr_constituents(all_expt_plates[id_qpcr_plate], all_expt_plates)

wells_by_id_assay = get_wells_by_id_assay(id_plate_constituents)

id_qpcr_datas = build_id_qpcr_datas_from_inst_data(id_plate_constituents, id_qpcr_inst_plate)

Wall time: 23 ms


In [7]:
md = dict((k, v) for k, v in id_plate_constituents.items() if '12' in k)


In [8]:
concs = dict((w, wc.get_id_template_attribute('concentration'))
                 for w, wc in md.items())


In [9]:
#all_expt_plates['A81_E214_1_ID']['A01']
#sorted(wells_by_id_assay[('Ab_pgaD_x.10_Aba38_Aba42',)])
id_qpcr_datas['A01']

IdQpcrData([('Ct', 5.587069988250732),
            ('∆NTC Ct', 0.0),
            ('Ct Call', False),
            ('Tm1', 79.1648941040039),
            ('Tm2', 74.78462219238281),
            ('Tm3', 62.46510314941406),
            ('Tm4', nan),
            ('Tm Specif', False),
            ('Tm NS', False),
            ('Tm PD', True)])

In [10]:
%%time

assays = {}
for assay in wells_by_id_assay:
    for a in assay:
        assays[a] = int(get_assay_amplicon_length(a))

mapping = create_qpcr_lc_mapping(all_expt_plates[lc_plate])
dilutions = get_lc_dilutions(all_expt_plates[lc_plate])
lc_datas = build_labchip_datas_from_inst_data(
    id_plate_constituents, lc_inst_plate, mapping, assays, dilutions)

Wall time: 847 ms


In [30]:
wells_by_id_assay.keys()

dict_keys([('Ca_rpb7_x.1_Cal04_Cal03',), ('Cg_rps0_x.1_Cgl03_Cgl04',), ('Ab_pgaD_x.10_Aba38_Aba42',), ('Ec_uidA_x.2_Eco64_Eco66',)])

In [11]:
%%time
qwells = wells_by_id_assay[('Ab_pgaD_x.10_Aba38_Aba42',)]
lcwells = []
for qw in qwells:
    if qw in mapping:
        lcwells.append(mapping[qw])
    else:
        lcwells.append(None)


nmt = NestedMasterTable.create_from_models(qwells,
                                           id_qpcr_plate,
                                           lcwells,
                                           lc_plate,
                                           id_plate_constituents,
                                           id_qpcr_datas,
                                           lc_datas)

Wall time: 998 µs


In [12]:
import pandas as pd
df = pd.DataFrame(nmt.rows)
df.loc[:, ['qPCR Well', 'Ct', '∆NTC Ct', 'Ct Call', 'Tm1', 'Tm2', 'Tm3', 'Tm4', 'Tm Specif','Tm NS', 'Tm PD']]

,qPCR Well,Ct,∆NTC Ct,Ct Call,Tm1,Tm2,Tm3,Tm4,Tm Specif,Tm NS,Tm PD
0,A03,5.578962,0.000000,False,75.469040,79.028015,61.917572,NaN,False,False,True
1,B03,12.346527,-6.767565,False,85.050888,78.617363,NaN,NaN,True,False,True
2,C03,13.985906,-8.406943,False,84.640236,NaN,NaN,NaN,True,False,False
3,D03,5.445383,0.133579,False,74.647736,NaN,NaN,NaN,False,False,True
4,E03,13.696951,-8.117989,False,84.640236,NaN,NaN,NaN,True,False,False
5,F03,15.137625,-9.558662,False,84.640236,NaN,NaN,NaN,True,False,False
6,G03,4.951226,0.627737,True,84.503357,89.704926,NaN,NaN,True,True,False
7,H03,NaN,NaN,False,84.640236,NaN,NaN,NaN,True,False,False
8,A07,19.546530,0.000000,False,74.784622,68.624863,NaN,NaN,False,False,True
9,B07,17.879883,1.666647,True,80.259964,85.324654,NaN,NaN,False,True,False


In [13]:
df

,qPCR Plate,qPCR Well,LC Plate,LC Well,PA Assay Name,PA Assay Conc.,PA Template Name,PA Template Conc.,PA Human Name,PA Human Conc.,...,Tm1,Tm2,Tm3,Tm4,Tm Specif,Tm NS,Tm PD,Specif ng/ul,NS ng/ul,PD ng/ul
0,A81_E214_1_ID,A03,20180103_A,A02,"(poolE1,)","(1.0,)",(),(),(),(),...,75.469040,79.028015,61.917572,NaN,False,False,True,0.000000,0.000000,0
1,A81_E214_1_ID,B03,20180103_A,B02,"(poolE1,)","(1.0,)",(),(),"(HgDNA_Promega287320,)","(3000.0,)",...,85.050888,78.617363,NaN,NaN,True,False,True,0.000000,0.000000,0
2,A81_E214_1_ID,C03,20180103_A,C02,"(poolE1,)","(1.0,)","(A.baumannii_1605,)","(5.0,)",(),(),...,84.640236,NaN,NaN,NaN,True,False,False,69.613989,0.000000,0
3,A81_E214_1_ID,D03,20180103_A,D02,"(poolE1,)","(1.0,)","(A.baumannii_1605,)","(5.0,)",(),(),...,74.647736,NaN,NaN,NaN,False,False,True,0.000000,0.000000,0
4,A81_E214_1_ID,E03,20180103_A,E02,"(poolE1,)","(1.0,)","(A.baumannii_1605,)","(5.0,)",(),(),...,84.640236,NaN,NaN,NaN,True,False,False,66.737663,0.000000,0
5,A81_E214_1_ID,F03,20180103_A,F02,"(poolE1,)","(1.0,)","(A.baumannii_1605,)","(5.0,)","(HgDNA_Promega287320,)","(3000.0,)",...,84.640236,NaN,NaN,NaN,True,False,False,49.436535,0.000000,0
6,A81_E214_1_ID,G03,20180103_A,G02,"(poolE1,)","(1.0,)","(A.baumannii_1605,)","(5.0,)","(HgDNA_Promega287320,)","(3000.0,)",...,84.503357,89.704926,NaN,NaN,True,True,False,56.234776,0.000000,0
7,A81_E214_1_ID,H03,20180103_A,H02,"(poolE1,)","(1.0,)","(A.baumannii_1605,)","(5.0,)","(HgDNA_Promega287320,)","(3000.0,)",...,84.640236,NaN,NaN,NaN,True,False,False,54.450344,0.000000,0
8,A81_E214_1_ID,A07,20180103_A,A04,"(Ab_pgaD_2.x_Aba26_Aba28,)","(0.4,)",(),(),(),(),...,74.784622,68.624863,NaN,NaN,False,False,True,0.000000,0.000000,0
9,A81_E214_1_ID,B07,20180103_A,B04,"(Ab_pgaD_2.x_Aba26_Aba28,)","(0.4,)",(),(),"(HgDNA_Promega287320,)","(3000.0,)",...,80.259964,85.324654,NaN,NaN,False,True,False,0.000000,0.000000,0
